In [1]:
import numpy as np
import pandas as pd

from transformers import AutoTokenizer
from sentence_transformers import SentenceTransformer, util
import torch
from tqdm import tqdm
import torch


In [4]:
PATH_COLLECTION_DATA = 'subtask4b_collection_data.pkl'
df_collection = pd.read_pickle(PATH_COLLECTION_DATA)

PATH_QUERY_TRAIN_DATA = 'subtask4b_query_tweets_train.tsv' #MODIFY PATH
PATH_QUERY_DEV_DATA = 'subtask4b_query_tweets_dev.tsv' #MODIFY PATH

df_query_train = pd.read_csv(PATH_QUERY_TRAIN_DATA, sep = '\t')
df_query_dev = pd.read_csv(PATH_QUERY_DEV_DATA, sep = '\t')

In [5]:
# Load pretrained sentence embedding model
model = SentenceTransformer('allenai-specter')  # Trained on scientific text

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/2.57k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/331 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/222k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/462k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [14]:
# Move model to GPU
model = model.to('cuda')

# Convert your input list to tensors and move to GPU if needed
tweet_text_dev = df_query_dev['tweet_text'].to_list()
# If using a Hugging Face model with .encode(), specify device if available
tweet_embeddings_dev = model.encode(
    tweet_text_dev,
    convert_to_tensor=True,
    show_progress_bar=True,
    device='cuda' if torch.cuda.is_available() else 'cpu'
)


Batches:   0%|          | 0/44 [00:00<?, ?it/s]

In [13]:
collection_text = []
for row in df_collection.itertuples():
    collection_text.append(row.title + " " + row.abstract if row.abstract else "")
collection_embeddings = model.encode(
    collection_text,
    convert_to_tensor=True,
    show_progress_bar=True,
    device='cuda' if torch.cuda.is_available() else 'cpu'
)

Batches:   0%|          | 0/242 [00:00<?, ?it/s]

In [18]:
# save embeddings
#np.save("paper_embeddings_v1.npy", collection_embeddings.cpu())


In [52]:
# Evaluate retrieved candidates using MRR@k
def get_performance_mrr(data, col_gold, col_pred, list_k = [1, 5, 10]):
    d_performance = {}
    for k in list_k:
        data["in_topx"] = data.apply(lambda x: (1/([i for i in x[col_pred][:k]].index(x[col_gold]) + 1) if x[col_gold] in [i for i in x[col_pred][:k]] else 0), axis=1)
        #performances.append(data["in_topx"].mean())
        d_performance[k] = data["in_topx"].mean()
    return d_performance


# get top cordids
def get_top_cordids_embeddings(tweet_embedding, k = 10):
    cos_scores = util.cos_sim(tweet_embedding, collection_embeddings)[0]
    top_results = torch.topk(cos_scores, k=k)
    return df_collection.iloc[top_results[1].cpu()]['cord_uid'].to_list()



top_cords_dev = []
for tweet_embedding in tweet_embeddings_dev:
    top_cords_dev.append(get_top_cordids_embeddings(tweet_embedding))

df_query_dev['embv1_topk'] = top_cords_dev



In [56]:
results_embedding_dev = get_performance_mrr(df_query_dev, 'cord_uid', 'embv1_topk')

print(f"Results on the dev set: {results_embedding_dev}")

Results on the dev set: {1: np.float64(0.26857142857142857), 5: np.float64(0.3258452380952381), 10: np.float64(0.33594982993197275)}
